In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import sys

In [ ]:
# Parameters
lr = 0.01
batch_size = 64
test_batch_size = 1000
epochs = 20
log_interval = 10
use_cuda = True if torch.cuda.is_available() else False

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [ ]:
model = Net()
if use_cuda:
    model.cuda()

In [ ]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
from torchvision import datasets, transforms

kwargs = {'num_workers': 1, 'pin_memory': True}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from torch.autograd import Variable

def train(epoch):
    model.train()
    train_loss = 0
    correct = 0
    sys.stdout.write('\nTrain Epoch: {} '.format(epoch))
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)        
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.data[0]
        pred = output.data.max(1)[1]
        correct += pred.eq(target.data).cpu().sum()
        
        if batch_idx % log_interval == 0:
            sys.stdout.write('-')

    train_loss /= len(train_loader)
    accuracy = correct/len(train_loader.dataset)
    
    sys.stdout.write('\nAvg loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            train_loss, correct, len(train_loader.dataset), 100. * accuracy))
    log_value('train_acc', accuracy, epoch)
    log_value('train_loss', train_loss, epoch)

In [ ]:
def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(test_loader) # loss function already averages over batch size
    accuracy = correct/len(test_loader.dataset)
    print('\nTest set: Avg loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), 100. * accuracy))
    log_value('test_acc', accuracy, epoch)
    log_value('test_loss', test_loss, epoch)

In [ ]:
from tensorboard_logger import configure, log_value
configure("runs/mnist_cnn_1-32-64")

In [ ]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)

In [ ]:
# Save model parameters
torch.save(model.state_dict(), 'model_32_64.t7')

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# How to show an image in the test dataset
img = test_loader.dataset[10][0][0].numpy()
imgplot = plt.imshow(img, cmap='gray')